In [10]:
import os
import cv2 as cv
import mediapipe as mp

# Initialize MediaPipe Hands
mp_hands = mp.solutions.hands
hands = mp_hands.Hands(static_image_mode=False, max_num_hands=2, min_detection_confidence=0.5)
mp_drawing = mp.solutions.drawing_utils

# Directory containing videos
video_dir = r"D:\libras-alunos"

# Process each video in the directory
for video_file in os.listdir(video_dir):
    if video_file.endswith('.mp4'):  # Check if the file is a video
        video_path = os.path.join(video_dir, video_file)
        cap = cv.VideoCapture(video_path)
        
        # Get video properties
        frame_width = int(cap.get(cv.CAP_PROP_FRAME_WIDTH))
        frame_height = int(cap.get(cv.CAP_PROP_FRAME_HEIGHT))
        fps = cap.get(cv.CAP_PROP_FPS)
        
        # Define the codec and create VideoWriter object
        out = cv.VideoWriter(f'{video_dir}/mask_{video_file}', cv.VideoWriter_fourcc(*'mp4v'), fps, (frame_width, frame_height), isColor=False)
        
        while cap.isOpened():
            ret, frame = cap.read()
            if not ret:
                break
            
            # Create a blank (black) mask with the same size as the frame
            mask = cv.cvtColor(frame, cv.COLOR_BGR2GRAY)
            mask.fill(0)  # Set all pixels to black
            
            # Convert the BGR image to RGB
            rgb_frame = cv.cvtColor(frame, cv.COLOR_BGR2RGB)
            
            # Process the frame and detect hands
            result = hands.process(rgb_frame)
            
            # Draw hand landmarks on the mask
            if result.multi_hand_landmarks:
                for hand_landmarks in result.multi_hand_landmarks:
                    mp_drawing.draw_landmarks(
                        mask, 
                        hand_landmarks, 
                        mp_hands.HAND_CONNECTIONS,
                        mp_drawing.DrawingSpec(color=(255, 255, 255), thickness=2),  # White connections
                        mp_drawing.DrawingSpec(color=(255, 255, 255), thickness=2)   # White points
                    )
            
            # Write the mask frame to the output video
            out.write(mask)
        
        # Release the video capture and writer objects
        cap.release()
        out.release()

# Release the MediaPipe Hands object
hands.close()


AttributeError: 'numpy.dtypes.BoolDType' object has no attribute 'bits'